# Image Augmentations for Model Training Data

## 1. Imports and GPU Check

In [ ]:
# Hide output of this cell
%%capture

# Install packages
!pip install fastcore fastai --upgrade
!pip install rasterio
!pip install geopandas

# Import packages
from fastai.vision.all import *
from PIL import Image
from torchvision.transforms import ToTensor
import albumentations as A
from sklearn.model_selection import train_test_split
from skimage.io import imread, imsave
import imageio
from skimage.transform import rotate
from skimage.util import random_noise
import cv2
import rasterio.features
import shapely.geometry
import geopandas as gpd
import numpy as np
import random
import os
import shutil
from tqdm import tqdm
from matplotlib import pyplot as plt
import glob
import pytz
from datetime import datetime
from google.colab import drive

## 2. Define Required Functions and Set Parameters

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

def informal_pixels_from_file(file):
  """Calculate share of pixels in a mask that belong to informal settlements."""
  with rasterio.open(file, 'r') as mask:
    mask = mask.read()
    settlement_pixels = np.count_nonzero(mask == 255)
    total_pixels = mask.size
    settlement_share = settlement_pixels / total_pixels
    return settlement_share

Mounted at /content/drive


## 2. Create Image Augmentations

### 2.1. Create undersampled training data to use as baseline for augmentations

In [ ]:
# Set type of imagery, mask, and tiles to augment, as well as root directory
imagery_type = "aerial"
mask_type = "buildings"
tile_type = "512_512 stride"
path = Path(f"/content/drive/MyDrive/Segmentation Data/{imagery_type}")

# Set paths to images and corresponding masks and retrieve a list of all masks
dir_img = f'{path}/image_tiles/"2019_10cm_RGB_BE_67"/{tile_type}'
dir_msk = f'{path}/{mask_type}_mask_tiles/"2019_10cm_RGB_BE_67"/{tile_type}'
lbl_names = get_image_files(dir_msk)

# Create a list of masks containing buildings
building_share = []
for fn in lbl_names:
  building_share.append(informal_pixels_from_file(fn))
index = 0
building_tiles = []
for mask in building_share:
  if mask > 0:
    building_tiles.append(index)
  index += 1

# Set paths for undersampled images and masks
dir_img_augm = f'{path}/image_tiles/"2019_10cm_RGB_BE_67"/512_512 undersampled'
dir_msk_augm = f'{path}/{mask_type}_mask_tiles/"2019_10cm_RGB_BE_67"/512_512 undersampled'

# Loop through all masks containing buildings and copy them, as well as the corresponding image tiles to a new folder
for tile in building_tiles:
  shutil.copyfile(fnames[tile], f"{dir_img_augm}/{str(fnames[index]).split('/')[-1]}")
  shutil.copyfile(lbl_names[tile], f"{dir_msk_augm}/{str(lbl_names[index]).split('/')[-1]}")

### 2.2. Create Augmentations

In [ ]:
## Retrieve a list of all image and mask tiles in the new undersampled folder and set path to final folder for augmented images and masks
fnames = get_image_files(dir_img_augm)
lbl_names = get_image_files(dir_msk_augm)
root = '/content/drive/MyDrive/Segmentation Data/aerial/augmented/8'

# Loop through both tested splits, folders for images and masks, and the train and validation folder
for split in [0.05, 0.2]:
  for tile in ['img', 'lbl']:
    for folder in ["train", "valid"]:
      # Create a variable with the directory to each of the new folders
      local_dir = f'{root}/{i}/{tile}/{folder}'
      # If they don't already exist, create the folders
      if not os.path.exists(local_dir):
          os.makedirs(local_dir)
      # Randomly select 5% and 20% of images and masks
      img_train, img_test, msk_train, msk_test = train_test_split(fnames, lbl_names, test_size = split, random_state = 42)

      # Copy selected images and masks to the 'valid' folder (they will remain untouched)
      for file in img_test:
        shutil.copyfile(file, f"{local_dir}/{str(file).split('/')[-1]}")
      for file in msk_test:
        shutil.copyfile(file, f"{local_dir}/{str(file).split('/')[-1]}")

      # Check whether there is a mask for each validation image
      if len(img_train) != len(msk_train):
        print(len(img_train), len(msk_train))

      # Open images and masks and append them to two new lists
      train_img = []
      train_msk = []
      
      for img_path in img_train:
        img = imread(img_path)
        train_img.append(img)
      for msk_path in msk_train:
        msk = imread(msk_path)
        train_msk.append(msk)

      # Convert lists of read-in images and masks to Numpy arrays to speed up augmentations 
      train_img = np.array(train_img)
      train_msk = np.array(train_msk)

      ## Create augmentations
      final_train_data = []
      final_target_train = []

      # Loop through all images and corresponding masks to be augmented and append them and their transformations to the two final lists
      for i in tqdm(range(train_img.shape[0])):
          final_train_data.append(train_img[i]) # original image
          final_train_data.append(rotate(train_img[i], angle = 90)) # 90 degree flipped
          final_train_data.append(np.fliplr(train_img[i])) # left-right flipped
          final_train_data.append(np.flipud(train_img[i])) # up-down flipped
          final_train_data.append(random_noise(train_img[i], var = 0.2**2)) # random noise added
          final_train_data.append(np.fliplr(rotate(train_img[i], angle = 90))) # rotated and left-right flipped
          final_train_data.append(np.flipud(rotate(train_img[i], angle = 90))) # rotated and up-down flipped

          final_target_train.append(train_msk[i]) # original mask
          final_target_train.append(rotate(train_msk[i], angle = 90)) # 90 degree flipped
          final_target_train.append(np.fliplr(train_msk[i])) # left-right flipped
          final_target_train.append(np.flipud(train_msk[i])) # up-down flipped
          final_target_train.append(train_msk[i]) # use orignal mask since position of buildings has not changed
          final_target_train.append(np.fliplr(rotate(train_msk[i], angle = 90))) # rotated and left-right flipped
          final_target_train.append(np.flipud(rotate(train_msk[i], angle = 90))) # rotated and up-down flipped

          index = 0
          # Store each augmented image and mask in the final augmentation folder
          for img in final_train_data:
            imageio.imwrite(f'{dir_augm}/img/train/{index}.png', img*255.astype(np.uint8))
            index += 1

          index = 0
          for msk in final_target_train:
            imageio.imwrite(f'{dir_augm}/lbl/train/{index}.png', msk*255).astype(np.uint8))
            index += 1

100%|██████████| 207/207 [00:43<00:00,  4.72it/s]
